In [1]:
import pandas as pd
# options for pandas 
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import natasha
import razdel
import re
import string
import annoy
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
   
    PER,
    Doc
)
from gensim.models import FastText, word2vec, ldamodel
from gensim import corpora
import nltk
from nltk.corpus import stopwords
stopwords_ru = stopwords.words('russian')
nltk.download('stopwords')
from pymorphy2 import MorphAnalyzer
from corus import load_lenta
import pyLDAvis
import pyLDAvis.gensim as gensimvis
path = 'lenta-ru-news.csv.gz'
records = load_lenta(path)
import warnings
warnings.filterwarnings('ignore')

/home/medic/python36/lib64/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """
[nltk_data] Downloading package stopwords to /home/medic/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Возьмем dataframe с прошлого урока со статьями сайта lenta.ru

In [2]:
lenta_ru = pd.read_pickle('lenta_ru_w2c.pickle')
lenta_ru = lenta_ru.drop(['lemmatize_title', 'vector_title', 'lemmaatize_title_tags_topic'], axis = 1)
lenta_ru

,tags,title,topic,text
0,Мир,Обнаружено самое счастливое место на планете,Путешествия,"Сотрудники социальной сети Instagram проанализировали поставленные пользователями смайлики, геолокации и хештеги и опубликовали итоги 2018 года. Об этом сообщается на официальном сайте Instagram. Таким образом, самой счастливой геолокацией Instagram признал Диснейленд в Токио, так как больше всего счастливых смайликов в 2018 году пользователи ставили именно под фотографиями из японского Диснейленда. Также эксперты назвали самый популярный фильтр для лица: им стал фильтр с сердечками на глазах. А, например, самыми часто используемыми хештегами в 2018 году были #metoo, #timesup и #marchforourlives. В ноябре сотрудники британской ассоциации потребителей Which? составили рейтинг самых безопасных стран для путешествий. Специалисты проанализировали 20 самых популярных туристических направлений по четырем критериям: уровень преступности, угроза здоровью, вероятность теракта и стихийных бедствий. Самой безопасной страной по всем параметрам стала Исландия."
1,Общество,Хакеры рассказали о планах Великобритании заминировать Севастополь,Мир,"Хакерская группировка Anonymous опубликовала новые документы о деятельности британского аналитического центра Integrity Initiative. Из материалов следует, что центр получает финансирование от некого Института государственного управления под руководством Кристофера Доннелли. Хакеры выложили в открытый доступ его паспорт, резюме, а также сведения о его связях с Министерством обороны Великобритании. Согласно документам, в 2014 году Доннелли выдвинул ряд предложений британским властям в связи с ситуацией в Крыму. В частности, он планировал заминировать Севастопольскую бухту, окружить полуостров войсками, а также уничтожить оставшиеся в Крыму самолеты «в знак серьезности намерений». После публикации заметки часть документов, на которые члены группировки ссылаются в своем расследовании, была удалена. Также хакеры утверждают, что Доннелли был инициатором расследования «российского вмешательства» в референдум о независимости в Каталонии. Он пригласил членов испанского отделения Integrity Initiative в качестве свидетелей. По данным группировки, Доннелли получал деньги на это от Министерства иностранных дел Великобритании. «Британская разведка платила собственным агентам за фальшивые доказательства вмешательства России в каталонский референдум, а затем приказала им солгать в парламенте с целью убедить их предпринять антироссийские шаги», — пишут хакеры. Члены Anonymous опубликовали только часть документов, связанных с деятельностью Кристофера Доннелли. Если расследование Великобритании в отношении Integrity Initiative не даст результатов, они пригрозили выложить новые доказательства того, что Доннелли тесно взаимодействует с британскими спецслужбами и использует свое положение для влияния на политику страны. 23 ноября Anonymous впервые опубликовали документы Integrity Initiative, которые содержат инструкции по борьбе с российской пропагандой и примеры дезинформации со стороны Москвы. Хакеры утверждают, что этот проект финансируется правительством Великобритании и работает сразу в нескольких странах, в том числе в Германии, Франции, Испании."
2,Coцсети,Российскую молодежь предложили обучать духовным ценностям в соцсетях,Интернет и СМИ,"Российская молодежь лучше усвоит духовные ценности, если распространять их через интернет и социальные сети. С такой инициативой выступил глава Федерального агентства по делам молодежи Александр Бугаев в ходе религиозного форума «Россия в будущее», передает радио «Говорит Москва». Бугаев отметил, что раньше с этой функцией справлялись плакаты, которые можно было повесить в учебных заведениях, однако в настоящее время этого недостаточно. «Намного эффективнее будет, если мы об этом хорошем, добром и ценном расскажем в социальных группах в интернете. При помощи мессенджеров, неформальных объединений молодежи, которые зачастую охватывают огромное количество людей», — сказал он. По словам чиновника, 

Посмотрим столько у нас тем в данной выборке

In [3]:
lenta_ru['topic'].value_counts()

Россия               80325
Мир                  68241
Экономика            39836
Спорт                32253
Культура             27033
Бывший СССР          26743
Наука и техника      26470
Интернет и СМИ       22209
Из жизни             13758
Дом                  10885
Силовые структуры    9737 
Ценности             3905 
Бизнес               3763 
Путешествия          3183 
69-я параллель       638  
Крым                 327  
Культпросвет         174  
                     100  
Легпром              57   
Библиотека           32   
Оружие               3    
МедНовости           1    
ЧМ-2014              1    
Сочи                 1    
Name: topic, dtype: int64

In [4]:
topic_list = ['Россия', 'Мир', 'Спорт', 'Культура', 'Экономика', 'Дом']
df_topics = lenta_ru[lenta_ru['topic'].isin(topic_list)]
df_topics = df_topics.sample(35000)

In [5]:
add_stop_words = ['год', 'который', 'это', 'сообщать', 'также', 'свой']
stopwords_ru.extend(add_stop_words)

возьмем метод из предыдущего урока для предобработки текста (убрать stopwords, знаки препинания, короткие слова и т.д.)

In [6]:
# method for cleaning text
morph = MorphAnalyzer()
def prep_text(text):
    text = text.lower()
    text = re.sub(r'@[\w]*', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'[\d]', '', text)
    text = ' '.join([word for word in text.split() if word not in stopwords_ru])
    output_text = []
    # lemmatize text
    for token in text.split():
        token = morph.normal_forms(token)[0]
        
        if token not in stopwords_ru:
            output_text.append(token)
    
    return output_text

In [7]:
df_topics['text_lemmatized'] = df_topics.apply(lambda x: prep_text(x['text']), axis = 1)
df_topics[5:9]

,tags,title,topic,text,text_lemmatized
202276,Все,Сборная Нидерландов одержала вторую победу на ЧМ-2010,Спорт,"Сборная Нидерландов одержала победу над командой Японии во втором групповом матче чемпионата мира по футболу. Встреча, состоявшаяся в Дурбане, закончилась со счетом 1:0 - единственный гол на 53-й минуте игры забил Весли Снейдер. Встреча проходила в равной борьбе, более того, в отдельных отрезках матча преимущество было на стороне японцев. В первом тайме, однако, командам не удалось создать ни одного опасного момента; было нанесено всего два удара в створ ворот. Второй тайм начался с атак сборной Голландии, одна из которых завершилась голом - после розыгрыша углового Робин ван Перси откинул мяч под удар Снейдеру. Японский голкипер задел мяч, но отбить не смог. В концовке матча вышедший на замену Афеллай имел два шанса увеличить преимущество голландцев, однако оба его выхода один на один нейтрализовал вратарь. Благодаря победе голландцы вышли в лидеры группы E с 6 очками. Позднее 19 июня состоится матч двух других команд группы - Камерун сыграет против Дании. У обеих команд пока нет набранных очков.","[сборная, нидерланды, одержать, победа, команда, япония, второй, групповой, матч, чемпионат, мир, футбол, встреча, состояться, дурбан, закончиться, счёт, единственный, гол, й, минута, игра, забить, вести, снейдёр, встреча, проходить, равный, борьба, отдельный, отрезок, матч, преимущество, сторона, японец, первый, тайм, однако, команда, удаться, создать, опасный, момент, нанести, удар, створ, ворота, второй, тайм, начаться, атака, сборная, голландия, завершиться, голый, розыгрыш, угловой, робин, ван, перси, откинуть, мяч, удар, снейдрать, японский, голкипер, задеть, мяч, отбить, смочь, концовка, матч, выйти, замена, афеллай, иметь, шанс, увеличить, преимущество, голландец, однако, оба, выход, нейтрализовать, вратарь, благодаря, победа, голландец, выйти, лидер, группа, e, очки, поздний, июнь, состояться, матч, команда, группа, камерун, ...]"
36393,Политика,КНДР заподозрили в фотографировании южнокорейской ПРО с беспилотника,Мир,"Северная Корея, предположительно, сфотографировала американскую ПРО THAAD, размещенную в Южной Корее, с помощью беспилотного летательного аппарата (БПЛА). Об этом сообщает The South China Morning Post. Военные обнаружили внутри беспилотника камеру Sony и несколько сотен фотографий ракетных установок THAAD и радарной установки, возведенных в городе Чонджу. Кроме того, на кадрах также были запечатлены городские районы, фермерские угодья и другие области страны. Как отмечает издание, беспилотники применяются Пхеньяном лишь несколько лет: впервые они были обнаружены лишь в 2014 году. «Мы придумаем способ справиться с северокорейскими БПЛА», — заявил неназванный сотрудник южнокорейского Генштаба.","[северный, корея, предположительно, сфотографировать, американский, thaad, разместить, южный, корея, помощь, беспилотный, летательный, аппарат, бпнуть, the, south, china, morning, post, военный, обнаружить, внутри, беспилотник, камера, sony, несколько, сотня, фотография, ракетный, установка, thaad, радарный, установка, возвести, город, чонджу, кроме, кадр, запечатлеть, городской, район, фермерский, угодье, область, страна, отмечать, издание, беспилотник, применяться, пхеньян, лишь, несколько, впервые, обнаружить, лишь, придумать, способ, справиться, северокорейский, бпнуть, заявить, неназванный, сотрудник, южнокорейский, генштаб]"
280007,Все,По поручению Кадырова выявлены нарушения прав человека,Россия,"По поручению премьер-министра Чечни Рамзана Кадырова была проведена проверка соблюдения прав человека в Грозненском следственном изоляторе и в колонии для осужденных в поселке Чернокозово. По словам чеченского уполномоченного по правам человека Норди Нухажиева, нарушения закона были выявлены в Грозненском СИЗО, сообщает агентство ""Интерфакс"". В частности, по его словам, в СИЗО ""подследственные, больные туберкулезом, психическими заболеваниями содержатся совместно со здоровыми людьми"". При этом Нухажиев отмет

In [8]:
df_topics.to_pickle('df_topics.pickle')

In [9]:
dictionary = corpora.Dictionary(df_topics['text_lemmatized'])
dictionary.filter_extremes(no_below = 10, no_above = 0.9, keep_n = None)

In [10]:
# vectorisation documents in dataframe
corpus = [dictionary.doc2bow(text) for text in df_topics['text_lemmatized']]
corpora.MmCorpus.serialize('lenta.model', corpus)

In [11]:
# training model lda with 20 tipics
lda = ldamodel.LdaModel(corpus, id2word = dictionary, num_topics = 20, chunksize = 70, update_every = 1, passes = 5 )

In [12]:
lda.show_topics(num_topics = 10, num_words = 10, formatted = True)

[(16,
  '0.077*"военный" + 0.045*"израиль" + 0.040*"великобритания" + 0.038*"операция" + 0.033*"израильский" + 0.033*"семья" + 0.031*"палестинский" + 0.031*"британский" + 0.028*"граница" + 0.027*"военнослужащий"'),
 (10,
  '0.012*"день" + 0.012*"роль" + 0.012*"алексей" + 0.012*"участие" + 0.011*"планироваться" + 0.011*"мероприятие" + 0.010*"имя" + 0.010*"стать" + 0.010*"пройти" + 0.010*"состояться"'),
 (4,
  '0.035*"человек" + 0.021*"произойти" + 0.019*"погибнуть" + 0.019*"результат" + 0.016*"находиться" + 0.016*"время" + 0.015*"пострадать" + 0.015*"город" + 0.014*"данные" + 0.011*"место"'),
 (17,
  '0.060*"проект" + 0.046*"строительство" + 0.041*"москва" + 0.041*"город" + 0.033*"объект" + 0.033*"мэр" + 0.030*"участок" + 0.029*"центр" + 0.029*"комплекс" + 0.025*"здание"'),
 (7,
  '0.041*"выборы" + 0.040*"партия" + 0.023*"депутат" + 0.022*"место" + 0.022*"пост" + 0.021*"список" + 0.020*"кандидат" + 0.020*"группа" + 0.019*"the" + 0.018*"лидер"'),
 (18,
  '0.095*"суд" + 0.040*"дело" + 0.0

Построим графическую схему полученных класстеров с помощью библитотеки <b>pyLDavis</b>

In [13]:
vis_data = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis_data)

Результат работы модели которую мы обучили похож на нашу изначальную подборку новостных тегов. Мы выбрали 5 групп тематик "'Россия', 'Мир', 'Спорт', 'Культура', 'Экономика', 'Дом'". На построенной схеме отчетливо можно выделить как минимум 4 кластера, отражает нашу первоначальную выбору из исходного датафрейма. 
Можно попробовать постоить еще одну lda модель, с количеством тем соотвествующим количеству выбранных нами из исходного набора данных.

Кроме этого, есть слова которые часто встречаются в текстах, но которые не несут никакого смысла. Можно попробовать добавить их в стоп-слова, и снова построить словарь и заново обучить модель.

In [17]:
lda_nextgen = ldamodel.LdaModel(corpus, id2word = dictionary, num_topics = len(topic_list), chunksize = 70, passes = 5 )
vis_data_nextgen = gensimvis.prepare(lda_nextgen, corpus, dictionary)
pyLDAvis.display(vis_data_nextgen)

#### Выводы по данному исследованию

Модель LDA (латентное замещение Дирихле) справляется с тематическим моделирование довольно не плохо.
Даже при наличии большего количества тем в параметрах модели, можно увидеть что более мелкие кластеры сливаются в больший кластер, тем самым отражая исходную картину разбивки по тематикам, существующую в исходном датасете.
Подход с тематическим разделением текстов LDA однозначно имеет место на сущестование при работе с большим количеством текстов.